# Batch-only Analysis

1. Exploratory Data Analysis
2. M1-M5 tests: Main Effects (model, country, license, map usage type, task type)
3. C1-C6 tests: Map Characteristics (graphical complexity, spatial aggregation level, map source, viz technique, symbol scaling, diagram structure)
4. I1-I4 tests: Interactions (model×map usage type, graphical complexity×map usage type, model×graphical complexity, model×spatial aggregation level)
5. P1 test: Clusters

In [1]:
import warnings
import pandas as pd
import numpy as np

from scipy.stats import shapiro, kruskal, mannwhitneyu, levene, t, sem, rankdata, chi2, kendalltau

from statsmodels.stats.multitest import multipletests
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

from scikit_posthocs import posthoc_dunn

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

warnings.filterwarnings('ignore')

df_batch = pd.read_csv('../../data/cleaned_data/data_batch_only.csv', index_col='answer_id')

df_batch.sample(5)

,model_name,provider_country,licence_type,map_code,graphical_complexity,viz_technique,symbol_scaling,diagram_structure,map_source,question_id,nuts_level,map_usage_type,task_type,test_mode,score
answer_id,,,,,,,,,,,,,,,
1549,Sonar,USA,paid,MULTI-G-INST-3,high,NaN,proportional symbols,uniform,statistical_office,109,country,reading,identify,batch,0.00
107,GPT-4o,USA,paid,MULTI-G-INST-2,high,NaN,proportional symbols,structural,statistical_office,107,country,interpretation,cause/effect,batch,4.75
946,Claude 3.5 Sonnet v2,USA,paid,MULTI-G-ATL-2,high,NaN,graduated symbols,structural,atlas,82,country,reading,identify,batch,4.90
426,GPT o3,USA,paid,MULTI-D-ATL-3,high,NaN,graduated symbols,uniform,atlas,138,region,reading,retrieve value,batch,5.00
1126,Claude 3.7 Sonnet,USA,paid,MULTI-D-ATL-1,high,NaN,graduated symbols,uniform,atlas,118,region,reading,identify,batch,4.50


In [2]:
df_batch.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1728 entries, 1 to 2304
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   model_name            1728 non-null   object 
 1   provider_country      1728 non-null   object 
 2   licence_type          1728 non-null   object 
 3   map_code              1728 non-null   object 
 4   graphical_complexity  1728 non-null   object 
 5   viz_technique         864 non-null    object 
 6   symbol_scaling        1188 non-null   object 
 7   diagram_structure     1188 non-null   object 
 8   map_source            1728 non-null   object 
 9   question_id           1728 non-null   int64  
 10  nuts_level            1728 non-null   object 
 11  map_usage_type        1728 non-null   object 
 12  task_type             1728 non-null   object 
 13  test_mode             1728 non-null   object 
 14  score                 1728 non-null   float64
dtypes: float64(1), int64(1), o

In [3]:
df_reading = df_batch[df_batch['map_usage_type'] == 'reading']
df_analysis = df_batch[df_batch['map_usage_type'] == 'analysis']
df_interpretation = df_batch[df_batch['map_usage_type'] == 'interpretation']

## 1. Exploratory Data Analysis

In [4]:
print("1. BASIC INFORMATION")
print("-" * 80)
print(f"Total observations: {len(df_batch)}")
print(f"Unique questions: {df_batch['question_id'].nunique()}")
print(f"Unique models: {df_batch['model_name'].nunique()}")
print(f"Models analyzed: {', '.join(sorted(df_batch['model_name'].unique()))}")

# Check if test_mode is all 'batch'
test_modes = df_batch['test_mode'].unique()
print(f"\nTest mode(s): {', '.join(test_modes)}")
if len(test_modes) > 1 or test_modes[0] != 'batch':
    print("WARNING: Dataset contains non-batch observations!")

1. BASIC INFORMATION
--------------------------------------------------------------------------------
Total observations: 1728
Unique questions: 144
Unique models: 12
Models analyzed: Claude 3.5 Sonnet v2, Claude 3.7 Sonnet, DeepSeek-R1, GPT o3, GPT-4o, Gemini 1.5 Pro, Gemma 3, Grok-3, MiniMax-01, Mistral Large, Qwen2.5-Max, Sonar

Test mode(s): batch


In [5]:
print("2. MODEL DISTRIBUTION")
print("-" * 80)
model_counts = df_batch['model_name'].value_counts().sort_index()
print("\nObservations per model:")
for model, count in model_counts.items():
    print(f"  {model}: {count}")

# Check balance
balance_ratio = model_counts.max() / model_counts.min()
print(f"\nBalance ratio (max/min): {balance_ratio:.2f}")
if balance_ratio > 2:
    print("WARNING: Imbalanced design - some models have >2x observations of others")
else:
    print("Design reasonably balanced across models")

2. MODEL DISTRIBUTION
--------------------------------------------------------------------------------

Observations per model:
  Claude 3.5 Sonnet v2: 144
  Claude 3.7 Sonnet: 144
  DeepSeek-R1: 144
  GPT o3: 144
  GPT-4o: 144
  Gemini 1.5 Pro: 144
  Gemma 3: 144
  Grok-3: 144
  MiniMax-01: 144
  Mistral Large: 144
  Qwen2.5-Max: 144
  Sonar: 144

Balance ratio (max/min): 1.00
Design reasonably balanced across models


In [6]:
print("3. ZERO VALUES ANALYSIS")
print("-" * 80)
zeros_total = (df_batch['score'] == 0).sum()
zeros_pct = (zeros_total / len(df_batch)) * 100
print(f"Zero values: {zeros_total} ({zeros_pct:.2f}%)")

# Zeros per model
print("Zero values by model:")
for model in sorted(df_batch['model_name'].unique()):
    model_data = df_batch[df_batch['model_name'] == model]
    zeros = (model_data['score'] == 0).sum()
    zeros_pct_model = (zeros / len(model_data)) * 100
    print(f"  {model}: {zeros} ({zeros_pct_model:.1f}%)")

print("\nNote: Zeros represent failed responses, not missing data.")
print("All analyses include zeros as legitimate failure outcomes.")

3. ZERO VALUES ANALYSIS
--------------------------------------------------------------------------------
Zero values: 458 (26.50%)
Zero values by model:
  Claude 3.5 Sonnet v2: 28 (19.4%)
  Claude 3.7 Sonnet: 21 (14.6%)
  DeepSeek-R1: 48 (33.3%)
  GPT o3: 29 (20.1%)
  GPT-4o: 44 (30.6%)
  Gemini 1.5 Pro: 34 (23.6%)
  Gemma 3: 45 (31.2%)
  Grok-3: 36 (25.0%)
  MiniMax-01: 47 (32.6%)
  Mistral Large: 36 (25.0%)
  Qwen2.5-Max: 41 (28.5%)
  Sonar: 49 (34.0%)

Note: Zeros represent failed responses, not missing data.
All analyses include zeros as legitimate failure outcomes.


In [7]:
print("4. DESCRIPTIVE STATISTICS")
print("-" * 80)

def descriptive_statistics(df, value_col, group_col=None, ci=0.95):
    def calc_stats(series, ci_level):
        n = len(series)
        mean_val = series.mean()
        sem_val = sem(series)
        ci_low, ci_high = t.interval(ci_level, n-1, loc=mean_val, scale=sem_val)
        stats_dict = {
            "M": mean_val,
            "95% CI": (ci_low, ci_high),
            "Mdn": series.median(),
            "SD": series.std(),
            "Range": (series.min(), series.max()),
            "Q1": series.quantile(0.25),
            "Q3": series.quantile(0.75),
            "N": n
        }
        return stats_dict

    if group_col is None:
        stats_overall = calc_stats(df[value_col], ci)
        print("\nOverall statistics:")
        print(f"  M = {stats_overall['M']:.3f}, 95% CI = ({stats_overall['95% CI'][0]:.2f}, {stats_overall['95% CI'][1]:.2f}), "
              f"Mdn = {stats_overall['Mdn']:.3f}, SD = {stats_overall['SD']:.3f}")
        print(f"  Range: [{stats_overall['Range'][0]:.1f}, {stats_overall['Range'][1]:.1f}]")
        print(f"  Q1 = {stats_overall['Q1']:.2f}, Q3 = {stats_overall['Q3']:.2f}")
    else:
        print(f"\nStatistics by '{group_col}':")
        for group in sorted(df[group_col].unique()):
            group_data = df[df[group_col] == group][value_col]
            stats_group = calc_stats(group_data, ci)
            print(f"{group}:")
            print(f"  M = {stats_group['M']:.2f}, 95% CI = ({stats_group['95% CI'][0]:.2f}, {stats_group['95% CI'][1]:.2f}), "
                  f"Mdn = {stats_group['Mdn']:.2f}, SD = {stats_group['SD']:.2f}, "
                  f"Range = [{stats_group['Range'][0]:.1f}, {stats_group['Range'][1]:.1f}], "
                  f"Q1 = {stats_group['Q1']:.2f}, Q3 = {stats_group['Q3']:.2f}, N = {stats_group['N']}")


print("Overall dataset:")
descriptive_statistics(df_batch, value_col='score', group_col='model_name')
print("-" * 80)
print("Reading subset:")
descriptive_statistics(df_reading, value_col='score', group_col='model_name')
print("-" * 80)
print("Analysis subset:")
descriptive_statistics(df_analysis, value_col='score', group_col='model_name')
print("-" * 80)
print("Interpretation subset:")
descriptive_statistics(df_interpretation, value_col='score', group_col='model_name')


4. DESCRIPTIVE STATISTICS
--------------------------------------------------------------------------------
Overall dataset:

Statistics by 'model_name':
Claude 3.5 Sonnet v2:
  M = 3.58, 95% CI = (3.27, 3.88), Mdn = 4.43, SD = 1.86, Range = [0.0, 5.0], Q1 = 3.43, Q3 = 5.00, N = 144
Claude 3.7 Sonnet:
  M = 3.94, 95% CI = (3.66, 4.21), Mdn = 4.53, SD = 1.68, Range = [0.0, 5.0], Q1 = 4.05, Q3 = 5.00, N = 144
DeepSeek-R1:
  M = 2.80, 95% CI = (2.46, 3.14), Mdn = 3.90, SD = 2.07, Range = [0.0, 5.0], Q1 = 0.00, Q3 = 4.50, N = 144
GPT o3:
  M = 3.51, 95% CI = (3.21, 3.82), Mdn = 4.30, SD = 1.84, Range = [0.0, 5.0], Q1 = 3.40, Q3 = 4.80, N = 144
GPT-4o:
  M = 2.98, 95% CI = (2.64, 3.32), Mdn = 4.00, SD = 2.05, Range = [0.0, 5.0], Q1 = 0.00, Q3 = 4.56, N = 144
Gemini 1.5 Pro:
  M = 3.29, 95% CI = (2.97, 3.61), Mdn = 4.10, SD = 1.93, Range = [0.0, 5.0], Q1 = 2.41, Q3 = 4.80, N = 144
Gemma 3:
  M = 2.98, 95% CI = (2.64, 3.32), Mdn = 4.05, SD = 2.06, Range = [0.0, 5.0], Q1 = 0.00, Q3 = 4.50, N = 

In [8]:
print("5. DISTRIBUTION TESTS")
print("-" * 80)
print("\nShapiro-Wilk normality tests:")

def shapiro_test_by_group(df, value_col, group_col=None, min_n=3):
    if group_col:
        print(f"Shapiro-Wilk test for '{value_col}' by '{group_col}':\n")
        for group in sorted(df[group_col].unique()):
            group_data = df[df[group_col] == group][value_col]
            n = len(group_data)
            if n >= min_n:
                stat, p = shapiro(group_data)
                normal_status = "normal" if p > 0.05 else "non-normal"
                print(f"  {group}: W = {stat:.4f}, p = {p:.4f} ({normal_status}, n={n})")
            else:
                print(f"  {group}: insufficient data for test (n={n})")

    # Overall test
    overall_data = df[value_col]
    stat_overall, p_overall = shapiro(overall_data)
    overall_status = "normal" if p_overall > 0.05 else "non-normal"
    print(f"\nOverall: W = {stat_overall:.4f}, p = {p_overall:.4f} ({overall_status}, n={len(overall_data)})\n")

    if p_overall < 0.05:
        print("Conclusion: Non-parametric tests required (Kruskal-Wallis, Mann-Whitney)")
    else:
        print("Conclusion: Parametric tests may be appropriate, but verify assumptions")

print("Overall dataset:")
shapiro_test_by_group(df_batch, value_col='score', group_col='model_name')
print("-" * 80)
print("Reading subset:")
shapiro_test_by_group(df_reading, value_col='score', group_col='model_name')
print("-" * 80)
print("Analysis subset:")
shapiro_test_by_group(df_analysis, value_col='score', group_col='model_name')
print("-" * 80)
print("Interpretation subset:")
shapiro_test_by_group(df_interpretation, value_col='score', group_col='model_name')

5. DISTRIBUTION TESTS
--------------------------------------------------------------------------------

Shapiro-Wilk normality tests:
Overall dataset:
Shapiro-Wilk test for 'score' by 'model_name':

  Claude 3.5 Sonnet v2: W = 0.7025, p = 0.0000 (non-normal, n=144)
  Claude 3.7 Sonnet: W = 0.6148, p = 0.0000 (non-normal, n=144)
  DeepSeek-R1: W = 0.7589, p = 0.0000 (non-normal, n=144)
  GPT o3: W = 0.7015, p = 0.0000 (non-normal, n=144)
  GPT-4o: W = 0.7487, p = 0.0000 (non-normal, n=144)
  Gemini 1.5 Pro: W = 0.7427, p = 0.0000 (non-normal, n=144)
  Gemma 3: W = 0.7256, p = 0.0000 (non-normal, n=144)
  Grok-3: W = 0.7086, p = 0.0000 (non-normal, n=144)
  MiniMax-01: W = 0.7379, p = 0.0000 (non-normal, n=144)
  Mistral Large: W = 0.7081, p = 0.0000 (non-normal, n=144)
  Qwen2.5-Max: W = 0.7779, p = 0.0000 (non-normal, n=144)
  Sonar: W = 0.7874, p = 0.0000 (non-normal, n=144)

Overall: W = 0.7372, p = 0.0000 (non-normal, n=1728)

Conclusion: Non-parametric tests required (Kruskal-Walli

In [9]:
print("6. HOMOGENEITY OF VARIANCE")
print("-" * 80)

def levene_test(df, value_col, group_col):
    groups = [df[df[group_col] == g][value_col].values for g in sorted(df[group_col].unique())]

    stat, p = levene(*groups)

    print(f"Levene's test for '{value_col}' by '{group_col}':")
    print(f"W = {stat:.4f}, p = {p:.4f}")

    if p < 0.05:
        print("Conclusion: Variances are heterogeneous - use Welch's ANOVA or non-parametric tests")
    else:
        print("Conclusion: Variances are homogeneous")

levene_test(df_batch, value_col='score', group_col='model_name')
levene_test(df_reading, value_col='score', group_col='model_name')
levene_test(df_analysis, value_col='score', group_col='model_name')
levene_test(df_interpretation, value_col='score', group_col='model_name')


6. HOMOGENEITY OF VARIANCE
--------------------------------------------------------------------------------
Levene's test for 'score' by 'model_name':
W = 2.7026, p = 0.0019
Conclusion: Variances are heterogeneous - use Welch's ANOVA or non-parametric tests
Levene's test for 'score' by 'model_name':
W = 0.8068, p = 0.6334
Conclusion: Variances are homogeneous
Levene's test for 'score' by 'model_name':
W = 1.5696, p = 0.1037
Conclusion: Variances are homogeneous
Levene's test for 'score' by 'model_name':
W = 2.4537, p = 0.0053
Conclusion: Variances are heterogeneous - use Welch's ANOVA or non-parametric tests


In [10]:
print("7. MODEL CHARACTERISTICS DISTRIBUTION")
print("-" * 80)
print("\nProvider country distribution:")
country_dist = df_batch.groupby(['model_name', 'provider_country']).size().unstack(fill_value=0)
print(country_dist)

print("\nLicense distribution:")
license_dist = df_batch.groupby(['model_name', 'licence_type']).size().unstack(fill_value=0)
print(license_dist)

7. MODEL CHARACTERISTICS DISTRIBUTION
--------------------------------------------------------------------------------

Provider country distribution:
provider_country      China  France  USA
model_name                              
Claude 3.5 Sonnet v2      0       0  144
Claude 3.7 Sonnet         0       0  144
DeepSeek-R1             144       0    0
GPT o3                    0       0  144
GPT-4o                    0       0  144
Gemini 1.5 Pro            0       0  144
Gemma 3                   0       0  144
Grok-3                    0       0  144
MiniMax-01              144       0    0
Mistral Large             0     144    0
Qwen2.5-Max             144       0    0
Sonar                     0       0  144

License distribution:
licence_type          free  paid
model_name                      
Claude 3.5 Sonnet v2     0   144
Claude 3.7 Sonnet        0   144
DeepSeek-R1            144     0
GPT o3                   0   144
GPT-4o                   0   144
Gemini 1.5 Pro       

In [11]:
print("8. TASK CHARACTERISTICS")
print("-" * 80)
print("\nMap usage types:")
print(df_batch['map_usage_type'].value_counts().sort_index())

print("\nGraphical complexity levels:")
print(df_batch['graphical_complexity'].value_counts().sort_index())

print("\nNUTS levels:")
print(df_batch['nuts_level'].value_counts().sort_index())

print("\nMap sources:")
print(df_batch['map_source'].value_counts().sort_index())

8. TASK CHARACTERISTICS
--------------------------------------------------------------------------------

Map usage types:
map_usage_type
analysis          576
interpretation    576
reading           576
Name: count, dtype: int64

Graphical complexity levels:
graphical_complexity
high    864
low     864
Name: count, dtype: int64

NUTS levels:
nuts_level
country    864
region     864
Name: count, dtype: int64

Map sources:
map_source
atlas                 864
statistical_office    864
Name: count, dtype: int64


## 2. Main effects

In [12]:
# M1: Model Ranking
# H0: All models achieve the same mean scores
# H1: There are differences in mean scores between AI models

print("M1: MODEL RANKING")
print("-" * 80)

# Kruskal-Wallis test (non-parametric, >2 groups)
model_groups = [df_batch[df_batch['model_name'] == model]['score'].values
                for model in sorted(df_batch['model_name'].unique())]
h_stat, p_val = kruskal(*model_groups)

# Effect size: Epsilon squared
n = len(df_batch)
k = len(model_groups)
epsilon_sq = (h_stat - k + 1) / (n - k)

print(f"Kruskal-Wallis H = {h_stat:.3f}, p = {p_val:.6f}")
print(f"Effect size (ε²) = {epsilon_sq:.3f}")
print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")

def format_p_value(p):
    if p < 0.001:
        return "< 0.001"
    elif p < 0.01:
        return "< 0.01"
    elif p < 0.05:
        return "< 0.05"
    else:
        return f"{p:.4f}"

# Post-hoc Dunn test with FDR correction
if p_val < 0.05:
    print("\nPost-hoc pairwise comparisons (Dunn test with FDR correction):")
    posthoc = posthoc_dunn(df_batch, val_col='score', group_col='model_name', p_adjust='fdr_bh')

    # Extract significant pairs
    sig_pairs = []
    models = posthoc.index.tolist()
    for i, model1 in enumerate(models):
        for j, model2 in enumerate(models):
            if i < j and posthoc.loc[model1, model2] < 0.05:
                sig_pairs.append({
                    'Model 1': model1,
                    'Model 2': model2,
                    'p-value': posthoc.loc[model1, model2]
                })

    if sig_pairs:
        sig_df = pd.DataFrame(sig_pairs).sort_values('p-value')
        sig_df['p-value_formatted'] = sig_df['p-value'].apply(format_p_value)

        print(f"\n{len(sig_pairs)} significant pairwise differences found:")
        print(sig_df.to_string(index=False))
    else:
        print("No significant pairwise differences after FDR correction")

model_means = df_batch.groupby('model_name')['score'].agg(['mean', 'median', 'std']).sort_values('mean',
                                                                                                   ascending=False)
print("\nModel Rankings (by mean score):")
for rank, (model, row) in enumerate(model_means.iterrows(), 1):
    print(f"{rank}. {model}: M={row['mean']:.3f}, Mdn={row['median']:.3f}, SD={row['std']:.3f}")

M1: MODEL RANKING
--------------------------------------------------------------------------------
Kruskal-Wallis H = 90.453, p = 0.000000
Effect size (ε²) = 0.046
Significant: YES

Post-hoc pairwise comparisons (Dunn test with FDR correction):

32 significant pairwise differences found:
             Model 1        Model 2      p-value p-value_formatted
   Claude 3.7 Sonnet          Sonar 1.352936e-10           < 0.001
   Claude 3.7 Sonnet    DeepSeek-R1 8.490153e-08           < 0.001
   Claude 3.7 Sonnet     MiniMax-01 1.014199e-07           < 0.001
   Claude 3.7 Sonnet    Qwen2.5-Max 7.254231e-07           < 0.001
   Claude 3.7 Sonnet         GPT-4o 2.173157e-06           < 0.001
   Claude 3.7 Sonnet        Gemma 3 2.236691e-06           < 0.001
Claude 3.5 Sonnet v2          Sonar 2.274644e-06           < 0.001
              Grok-3          Sonar 2.119598e-04           < 0.001
              GPT o3          Sonar 2.119598e-04           < 0.001
Claude 3.5 Sonnet v2    DeepSeek-R1 2.845

In [13]:
# M2: Provider Country
# H0: Models from different countries achieve the same mean scores
# H1: Mean scores differ by country of origin

print("M2: PROVIDER COUNTRY")
print("-" * 80)

country_counts = df_batch['provider_country'].value_counts()
print(f"Country distribution: {dict(country_counts)}")

# All countries have n≥144, proceed with test
country_groups = [df_batch[df_batch['provider_country'] == country]['score'].values
                  for country in sorted(df_batch['provider_country'].unique())]
h_stat, p_val = kruskal(*country_groups)

n = len(df_batch)
k = len(country_groups)
epsilon_sq = (h_stat - k + 1) / (n - k)

print(f"\nKruskal-Wallis H = {h_stat:.3f}, p = {p_val:.6f}")
print(f"Effect size (ε²) = {epsilon_sq:.3f}")
print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")

country_means = df_batch.groupby('provider_country')['score'].agg(['mean', 'std', 'count']).sort_values('mean',
                                                                                                        ascending=False)
print("\nCountry performance:")
print(country_means)

if p_val < 0.05:
    posthoc = posthoc_dunn(df_batch, val_col='score', group_col='provider_country', p_adjust='fdr_bh')
    print("\nPost-hoc comparisons:")
    print(posthoc)


M2: PROVIDER COUNTRY
--------------------------------------------------------------------------------
Country distribution: {'USA': 1152, 'China': 432, 'France': 144}

Kruskal-Wallis H = 21.939, p = 0.000017
Effect size (ε²) = 0.012
Significant: YES

Country performance:
                      mean       std  count
provider_country                           
France            3.328299  1.979374    144
USA               3.283247  1.967697   1152
China             2.876910  2.037589    432

Post-hoc comparisons:
           China    France       USA
China   1.000000  0.004172  0.000017
France  0.004172  1.000000  0.718178
USA     0.000017  0.718178  1.000000


In [14]:
# M3: License Type
# H0: Paid and free models achieve the same mean scores
# H1: Mean scores differ between paid and free models

print("M3: LICENSE TYPE")
print("-" * 80)

license_counts = df_batch['licence_type'].value_counts()
print(f"License distribution: {dict(license_counts)}")

# Mann-Whitney U test (2 groups)
paid = df_batch[df_batch['licence_type'] == 'paid']['score'].values
free = df_batch[df_batch['licence_type'] == 'free']['score'].values

u_stat, p_val = mannwhitneyu(paid, free, alternative='two-sided')

# Effect size: rank biserial correlation
r = 1 - (2 * u_stat) / (len(paid) * len(free))

print(f"\nMann-Whitney U = {u_stat:.3f}, p = {p_val:.6f}")
print(f"Effect size (rank-biserial r) = {r:.3f}")
print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")

license_means = df_batch.groupby('licence_type')['score'].agg(['mean', 'median', 'std', 'count'])
print("\nLicense performance:")
print(license_means)

M3: LICENSE TYPE
--------------------------------------------------------------------------------
License distribution: {'paid': 1152, 'free': 576}

Mann-Whitney U = 382221.000, p = 0.000000
Effect size (rank-biserial r) = -0.152
Significant: YES

License performance:
                  mean  median       std  count
licence_type                                   
free          2.901780    4.00  2.042590    576
paid          3.327235    4.25  1.953572   1152


In [15]:
# M4: Map Usage Type
# H0: Map usage types do not affect mean scores
# H1: Mean scores differ between map usage types

print("M4: MAP USAGE TYPE")
print("-" * 80)

category_counts = df_batch['map_usage_type'].value_counts()
print(f"Category distribution: {dict(category_counts)}")

category_groups = [df_batch[df_batch['map_usage_type'] == cat]['score'].values
                   for cat in sorted(df_batch['map_usage_type'].unique())]
h_stat, p_val = kruskal(*category_groups)

n = len(df_batch)
k = len(category_groups)
epsilon_sq = (h_stat - k + 1) / (n - k)

print(f"\nKruskal-Wallis H = {h_stat:.2f}, p = {p_val:.6f}")
print(f"Effect size (ε²) = {epsilon_sq:.3f}")
print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")

category_means = df_batch.groupby('map_usage_type')['score'].agg(['mean', 'median', 'std']).sort_values('mean',
                                                                                                           ascending=False)
print("\nMap usage type performance:")
print(category_means)

if p_val < 0.05:
    posthoc = posthoc_dunn(df_batch, val_col='score', group_col='map_usage_type', p_adjust='fdr_bh')
    print("\nPost-hoc comparisons:")
    print(posthoc)

M4: MAP USAGE TYPE
--------------------------------------------------------------------------------
Category distribution: {'reading': 576, 'analysis': 576, 'interpretation': 576}

Kruskal-Wallis H = 15.19, p = 0.000503
Effect size (ε²) = 0.008
Significant: YES

Map usage type performance:
                    mean  median       std
map_usage_type                            
interpretation  3.693316    4.05  1.385971
analysis        3.033247    4.10  2.012549
reading         2.829688    4.50  2.356468

Post-hoc comparisons:
                analysis  interpretation   reading
analysis        1.000000        0.004564  0.000716
interpretation  0.004564        1.000000  0.477183
reading         0.000716        0.477183  1.000000


In [16]:
# M5: Task Type
# H0: There are no significant differences in model performance across different task types.
# H1: At least one task type differs significantly in performance compared to the others.

print("M5: TASK TYPE")
print("-" * 80)

def report_significant_pairs(posthoc_df, alpha_levels=[0.05, 0.01, 0.001]):
    sig_pairs = []

    for i, row in enumerate(posthoc_df.index):
        for j, col in enumerate(posthoc_df.columns):
            if j <= i:  # skip duplicates and diagonal
                continue
            p = posthoc_df.loc[row, col]
            if p < alpha_levels[0]:
                if p < alpha_levels[2]:
                    level = "< 0.001"
                elif p < alpha_levels[1]:
                    level = "< 0.01"
                else:
                    level = "< 0.05"
                sig_pairs.append((row, col, p, level))

    if not sig_pairs:
        print("No statistically significant differences found.")
    else:
        print("Significant post-hoc differences:")
        for a, b, p, lvl in sorted(sig_pairs, key=lambda x: x[2]):
            print(f"  {a} vs {b} — p = {p:.6f} ({lvl})")

mode_counts = df_batch['task_type'].value_counts()
print(f"Task type distribution:\n{mode_counts}\n")

# Check for small groups
if mode_counts.min() >= 20:
    mode_groups = [df_batch[df_batch['task_type'] == mode]['score'].values
                   for mode in sorted(df_batch['task_type'].unique())]
    h_stat, p_val = kruskal(*mode_groups)

    n = len(df_batch)
    k = len(mode_groups)
    epsilon_sq = (h_stat - k + 1) / (n - k)

    print(f"Kruskal-Wallis H = {h_stat:.3f}, p = {p_val:.6f}")
    print(f"Effect size (ε²) = {epsilon_sq:.3f}")
    print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")

    mode_means = df_batch.groupby('task_type')['score'].agg(['mean', 'median', 'std', 'count']).sort_values('mean',
                                                                                                                ascending=False)
    print("\nTask type performance:")
    print(mode_means)

    if p_val < 0.05:
        posthoc = posthoc_dunn(df_batch, val_col='score', group_col='task_type', p_adjust='fdr_bh')
        print("\nPost-hoc comparisons (showing p < 0.05):")
        print(posthoc)
        report_significant_pairs(posthoc)
else:
    print("WARNING: Some groups have n < 20. Consider grouping or skip test.")

M5: TASK TYPE
--------------------------------------------------------------------------------
Task type distribution:
task_type
identify          192
locate            192
retrieve value    192
compare           192
cluster           192
associate         192
interpret         192
cause/effect      192
predict           192
Name: count, dtype: int64

Kruskal-Wallis H = 100.070, p = 0.000000
Effect size (ε²) = 0.054
Significant: YES

Task type performance:
                    mean  median       std  count
task_type                                        
cause/effect    3.847656   4.050  1.088184    192
predict         3.818750   4.250  1.402611    192
identify        3.590104   4.900  2.144970    192
interpret       3.413542   4.000  1.585231    192
compare         3.313932   4.300  1.928545    192
associate       3.244922   4.100  1.930893    192
locate          2.658854   4.300  2.365729    192
cluster         2.540885   3.825  2.092346    192
retrieve value  2.240104   0.000  2.357

## 3. Map Characteristics

In [17]:
def mann_whitney_analysis(df, group_col, value_col, group1=None, group2=None):
    unique_groups = df[group_col].unique()

    if group1 is None or group2 is None:
        if len(unique_groups) != 2:
            raise ValueError(f"{group_col} must have exactly two unique groups if group1/group2 are not specified.")
        group1, group2 = unique_groups

    group1_values = df[df[group_col] == group1][value_col].values
    group2_values = df[df[group_col] == group2][value_col].values

    # Mann-Whitney U test
    u_stat, p_val = mannwhitneyu(group1_values, group2_values, alternative='two-sided')

    # Rank-biserial effect size
    r = 1 - (2 * u_stat) / (len(group1_values) * len(group2_values))

    # Descriptive stats
    group_stats = df.groupby(group_col)[value_col].agg(['mean', 'median', 'std', 'count'])

    # Output
    print(f"\n{group_col} distribution: {dict(df[group_col].value_counts())}")
    print(f"\nMann-Whitney U test between '{group1}' and '{group2}':")
    print(f"U = {u_stat:.3f}, p = {p_val:.6f}")
    print(f"Effect size (rank-biserial r) = {r:.3f}")
    print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")
    print("\nDescriptive statistics:")
    print(group_stats)

In [18]:
# C1: Map Graphical Complexity
# H0: Map graphical complexity does not affect mean scores
# H1: Mean scores differ between low-complex and high-complex maps

print("C1: MAP GRAPHICAL COMPLEXITY")
print("-" * 80)

print("\nOverall dataset:")
mann_whitney_analysis(df_batch, group_col='graphical_complexity', value_col='score')
print("-" * 80)
print("Reading subset:")
mann_whitney_analysis(df_reading, group_col='graphical_complexity', value_col='score')
print("-" * 80)
print("Analysis subset:")
mann_whitney_analysis(df_analysis, group_col='graphical_complexity', value_col='score')
print("-" * 80)
print("Interpretation subset:")
mann_whitney_analysis(df_interpretation, group_col='graphical_complexity', value_col='score')

C1: MAP GRAPHICAL COMPLEXITY
--------------------------------------------------------------------------------

Overall dataset:

graphical_complexity distribution: {'low': 864, 'high': 864}

Mann-Whitney U test between 'low' and 'high':
U = 420971.000, p = 0.000003
Effect size (rank-biserial r) = -0.128
Significant: YES

Descriptive statistics:
                          mean  median       std  count
graphical_complexity                                   
high                  2.930787     4.0  2.086586    864
low                   3.440046     4.3  1.861930    864
--------------------------------------------------------------------------------
Reading subset:

graphical_complexity distribution: {'low': 288, 'high': 288}

Mann-Whitney U test between 'low' and 'high':
U = 48043.500, p = 0.000514
Effect size (rank-biserial r) = -0.158
Significant: YES

Descriptive statistics:
                          mean  median       std  count
graphical_complexity                                   
hi

In [19]:
# C2 Spatial aggregation level
# H0: Territorial level does not affect mean scores
# H1: Mean scores differ between country and region level

print("C2: SPATIAL AGGREGATION LEVEL")
print("-" * 80)

print("\nOverall dataset:")
mann_whitney_analysis(df_batch, group_col='nuts_level', value_col='score')
print("-" * 80)
print("Reading subset:")
mann_whitney_analysis(df_reading, group_col='nuts_level', value_col='score')
print("-" * 80)
print("Analysis subset:")
mann_whitney_analysis(df_analysis, group_col='nuts_level', value_col='score')
print("-" * 80)
print("Interpretation subset:")
mann_whitney_analysis(df_interpretation, group_col='nuts_level', value_col='score')

C2: SPATIAL AGGREGATION LEVEL
--------------------------------------------------------------------------------

Overall dataset:

nuts_level distribution: {'country': 864, 'region': 864}

Mann-Whitney U test between 'country' and 'region':
U = 392809.500, p = 0.055964
Effect size (rank-biserial r) = -0.052
Significant: NO

Descriptive statistics:
                mean  median       std  count
nuts_level                                   
country     3.276736    4.10  1.954428    864
region      3.094097    4.05  2.028290    864
--------------------------------------------------------------------------------
Reading subset:

nuts_level distribution: {'country': 288, 'region': 288}

Mann-Whitney U test between 'country' and 'region':
U = 46759.000, p = 0.005201
Effect size (rank-biserial r) = -0.127
Significant: YES

Descriptive statistics:
                mean  median       std  count
nuts_level                                   
country     3.045833     4.5  2.326289    288
region      

In [20]:
# C3: Map Source
# H0: Map source does not affect mean scores
# H1: Mean scores differ by map source

print("C3: MAP SOURCE")
print("-" * 80)

print("\nOverall dataset:")
mann_whitney_analysis(df_batch, group_col='map_source', value_col='score')
print("-" * 80)
print("Reading subset:")
mann_whitney_analysis(df_reading, group_col='map_source', value_col='score')
print("-" * 80)
print("Analysis subset:")
mann_whitney_analysis(df_analysis, group_col='map_source', value_col='score')
print("-" * 80)
print("Interpretation subset:")
mann_whitney_analysis(df_interpretation, group_col='map_source', value_col='score')

C3: MAP SOURCE
--------------------------------------------------------------------------------

Overall dataset:

map_source distribution: {'atlas': 864, 'statistical_office': 864}

Mann-Whitney U test between 'atlas' and 'statistical_office':
U = 365969.500, p = 0.477000
Effect size (rank-biserial r) = 0.020
Significant: NO

Descriptive statistics:
                        mean  median       std  count
map_source                                           
atlas               3.105208     4.1  2.053629    864
statistical_office  3.265625     4.1  1.928775    864
--------------------------------------------------------------------------------
Reading subset:

map_source distribution: {'atlas': 288, 'statistical_office': 288}

Mann-Whitney U test between 'atlas' and 'statistical_office':
U = 38704.500, p = 0.143587
Effect size (rank-biserial r) = 0.067
Significant: NO

Descriptive statistics:
                        mean  median       std  count
map_source                                

In [21]:
def analyze_categorical_effect(df, group_col, value_col='score', min_group_size=20, do_posthoc=True):
    df_sub = df[df[group_col].notna() & (df[group_col] != '')].copy()
    print(f"Available observations: {len(df_sub)}")

    if len(df_sub) == 0:
        print("No data available for analysis.")
        return

    group_counts = df_sub[group_col].value_counts()
    print(f"{group_col} distribution:\n{group_counts}\n")

    if len(group_counts) < 2 or group_counts.min() < min_group_size:
        print(f"Insufficient data (need ≥{min_group_size} per group).")
        return

    groups = [df_sub[df_sub[group_col] == g][value_col].values for g in sorted(df_sub[group_col].unique())]

    if len(groups) == 2:
        u_stat, p_val = mannwhitneyu(groups[0], groups[1], alternative='two-sided')
        r = 1 - (2 * u_stat) / (len(groups[0]) * len(groups[1]))
        print(f"Mann–Whitney U = {u_stat:.3f}, p = {p_val:.6f}")
        print(f"Effect size (rank-biserial r) = {r:.3f}")
        print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")
    else:
        h_stat, p_val = kruskal(*groups)
        n, k = len(df_sub), len(groups)
        epsilon_sq = (h_stat - k + 1) / (n - k)
        print(f"Kruskal–Wallis H = {h_stat:.3f}, p = {p_val:.6f}")
        print(f"Effect size (ε²) = {epsilon_sq:.3f}")
        print(f"Significant: {'YES' if p_val < 0.05 else 'NO'}")

        if p_val < 0.05 and do_posthoc:
            posthoc = posthoc_dunn(df_sub, val_col=value_col, group_col=group_col, p_adjust='fdr_bh')
            print("\nPost-hoc Dunn test (adjusted p-values < 0.05):")
            sig_pairs = []
            for i, row in enumerate(posthoc.index):
                for j, col in enumerate(posthoc.columns):
                    if j <= i:
                        continue
                    p = posthoc.loc[row, col]
                    if p < 0.05:
                        sig_pairs.append((row, col, p))
            if not sig_pairs:
                print("No significant post-hoc differences.")
            else:
                for a, b, p in sorted(sig_pairs, key=lambda x: x[2]):
                    print(f"  {a} vs {b} — p = {p:.6f}")

    desc = df_sub.groupby(group_col)[value_col].agg(['mean', 'median', 'std', 'count']).sort_values('mean', ascending=False)
    print("\nDescriptive statistics:")
    print(desc)

In [22]:
# C4: MAP TYPE
# H0: There is no effect of the map type on model performance.
# H1: There is a significant effect of map type.

print("C4: MAP TYPE")
print("-" * 80)

print("\nOverall dataset:")
analyze_categorical_effect(df_batch, 'viz_technique', min_group_size=10)
print("-" * 80)
print("Reading subset:")
analyze_categorical_effect(df_reading, 'viz_technique', min_group_size=10)
print("-" * 80)
print("Analysis subset:")
analyze_categorical_effect(df_analysis, 'viz_technique', min_group_size=10)
print("-" * 80)
print("Interpreting subset:")
analyze_categorical_effect(df_interpretation, 'viz_technique', min_group_size=10)

C4: MAP TYPE
--------------------------------------------------------------------------------

Overall dataset:
Available observations: 864
viz_technique distribution:
viz_technique
choropleth             324
point-based symbols    324
cartogram              216
Name: count, dtype: int64

Kruskal–Wallis H = 11.159, p = 0.003775
Effect size (ε²) = 0.011
Significant: YES

Post-hoc Dunn test (adjusted p-values < 0.05):
  cartogram vs choropleth — p = 0.002525

Descriptive statistics:
                         mean  median       std  count
viz_technique                                         
cartogram            3.789352    4.35  1.634491    216
point-based symbols  3.439815    4.30  1.877706    324
choropleth           3.207407    4.10  1.955173    324
--------------------------------------------------------------------------------
Reading subset:
Available observations: 288
viz_technique distribution:
viz_technique
choropleth             108
point-based symbols    108
cartogram         

In [23]:
# C5: SYMBOL SCALING
# H0: There is no effect of the symbol scaling on model performance.
# H1: There is a significant effect of symbol scaling.

print("C5: SYMBOL SCALING")
print("-" * 80)

print("\nOverall dataset:")
analyze_categorical_effect(df_batch, 'symbol_scaling')
print("-" * 80)
print("Reading subset:")
analyze_categorical_effect(df_reading, 'symbol_scaling')
print("-" * 80)
print("Analysis subset:")
analyze_categorical_effect(df_analysis, 'symbol_scaling')
print("-" * 80)
print("Interpreting subset:")
analyze_categorical_effect(df_interpretation, 'symbol_scaling')

C5: SYMBOL SCALING
--------------------------------------------------------------------------------

Overall dataset:
Available observations: 1188
symbol_scaling distribution:
symbol_scaling
proportional symbols    648
graduated symbols       540
Name: count, dtype: int64

Mann–Whitney U = 164155.500, p = 0.062328
Effect size (rank-biserial r) = 0.062
Significant: NO

Descriptive statistics:
                          mean  median       std  count
symbol_scaling                                         
proportional symbols  3.208873    4.10  1.965625    648
graduated symbols     2.902500    4.05  2.123198    540
--------------------------------------------------------------------------------
Reading subset:
Available observations: 396
symbol_scaling distribution:
symbol_scaling
proportional symbols    216
graduated symbols       180
Name: count, dtype: int64

Mann–Whitney U = 17387.000, p = 0.054807
Effect size (rank-biserial r) = 0.106
Significant: NO

Descriptive statistics:
         

In [24]:
# C6: DIAGRAM STRUCTURE
# H0: There is no effect of the diagram structure on model performance.
# H1: There is a significant effect of diagram structure.

print("C6: DIAGRAM STRUCTURE")
print("-" * 80)

print("\nOverall dataset:")
analyze_categorical_effect(df_batch, 'diagram_structure')
print("-" * 80)
print("Reading subset:")
analyze_categorical_effect(df_reading, 'diagram_structure')
print("-" * 80)
print("Analysis subset:")
analyze_categorical_effect(df_analysis, 'diagram_structure')
print("-" * 80)
print("Interpreting subset:")
analyze_categorical_effect(df_interpretation, 'diagram_structure')

C6: DIAGRAM STRUCTURE
--------------------------------------------------------------------------------

Overall dataset:
Available observations: 1188
diagram_structure distribution:
diagram_structure
uniform       648
structural    540
Name: count, dtype: int64

Mann–Whitney U = 181197.000, p = 0.281950
Effect size (rank-biserial r) = -0.036
Significant: NO

Descriptive statistics:
                       mean  median       std  count
diagram_structure                                   
structural         3.109630   4.075  2.059428    540
uniform            3.036265   4.025  2.031297    648
--------------------------------------------------------------------------------
Reading subset:
Available observations: 396
diagram_structure distribution:
diagram_structure
uniform       216
structural    180
Name: count, dtype: int64

Mann–Whitney U = 18489.500, p = 0.374078
Effect size (rank-biserial r) = 0.049
Significant: NO

Descriptive statistics:
                       mean  median       std

## 4. Interactions

In [25]:
def scheirer_ray_hare_test(df, value_col, factor1_col, factor2_col):
    df = df.copy()
    df['rank'] = rankdata(df[value_col])

    n = len(df)
    a = df[factor1_col].nunique()
    b = df[factor2_col].nunique()

    grand_mean_rank = df['rank'].mean()

    ss_factor1 = 0
    for level in df[factor1_col].unique():
        subset = df[df[factor1_col] == level]
        n_level = len(subset)
        mean_rank = subset['rank'].mean()
        ss_factor1 += n_level * (mean_rank - grand_mean_rank)**2

    ss_factor2 = 0
    for level in df[factor2_col].unique():
        subset = df[df[factor2_col] == level]
        n_level = len(subset)
        mean_rank = subset['rank'].mean()
        ss_factor2 += n_level * (mean_rank - grand_mean_rank)**2

    ss_total = np.sum((df['rank'] - grand_mean_rank)**2)
    ss_interaction = ss_total - ss_factor1 - ss_factor2
    ms_total = ss_total / (n - 1)

    H_factor1 = ss_factor1 / ms_total
    H_factor2 = ss_factor2 / ms_total
    H_interaction = ss_interaction / ms_total

    df_factor1 = a - 1
    df_factor2 = b - 1
    df_interaction = (a - 1) * (b - 1)

    p_factor1 = 1 - chi2.cdf(H_factor1, df_factor1)
    p_factor2 = 1 - chi2.cdf(H_factor2, df_factor2)
    p_interaction = 1 - chi2.cdf(H_interaction, df_interaction)

    return {
        'model_effect': {'H': H_factor1, 'df': df_factor1, 'p': p_factor1},
        f'{factor2_col}_effect': {'H': H_factor2, 'df': df_factor2, 'p': p_factor2},
        'interaction': {'H': H_interaction, 'df': df_interaction, 'p': p_interaction}
    }


def permutation_interaction_test(df, value_col, factor1_col, factor2_col, n_permutations=10000):
    observed_stat = 0
    for level in df[factor2_col].unique():
        subset = df[df[factor2_col] == level]
        groups = [subset[subset[factor1_col] == m][value_col].values
                  for m in subset[factor1_col].unique()]
        h, _ = kruskal(*groups)
        observed_stat += h

    permuted_stats = []
    for _ in range(n_permutations):
        df_perm = df.copy()
        df_perm[value_col] = np.random.permutation(df_perm[value_col].values)

        perm_stat = 0
        for level in df_perm[factor2_col].unique():
            subset = df_perm[df_perm[factor2_col] == level]
            groups = [subset[subset[factor1_col] == m][value_col].values
                      for m in subset[factor1_col].unique()]
            h, _ = kruskal(*groups)
            perm_stat += h
        permuted_stats.append(perm_stat)

    p_value = np.mean(np.array(permuted_stats) >= observed_stat)

    return {'statistic': observed_stat, 'p_value': p_value}


def test_ranking_consistency(df, value_col, factor1_col, factor2_col):
    pivot = df.pivot_table(values=value_col, index=factor1_col,
                           columns=factor2_col, aggfunc='mean')

    rankings = pivot.rank(ascending=False)
    categories = rankings.columns.tolist()
    results = []

    for i, cat1 in enumerate(categories):
        for cat2 in categories[i+1:]:
            tau, p = kendalltau(rankings[cat1], rankings[cat2])
            results.append({
                'category1': cat1,
                'category2': cat2,
                'kendall_tau': tau,
                'p_value': p,
                'agreement': 'high' if tau > 0.7 else 'moderate' if tau > 0.4 else 'low'
            })

    return pd.DataFrame(results), rankings

In [26]:
# I1: Model x Map Usage Type intersection
# H0: No interaction effect between Model and Map Usage Type.
# H1: An interaction effect exists between Model and Map Usage Type.

print("I1: MODEL × MAP USAGE TYPE INTERACTION")
print("-" * 80)

print("Scheirer-Ray-Hare test:")
results = scheirer_ray_hare_test(df_batch, 'score', 'model_name', 'map_usage_type')
print(f"Model effect: H = {results['model_effect']['H']:.3f}, p = {results['model_effect']['p']:.6f}")
print(f"Usage type effect: H = {results['map_usage_type_effect']['H']:.3f}, p = {results['map_usage_type_effect']['p']:.6f}")
print(f"Interaction: H = {results['interaction']['H']:.3f}, p = {results['interaction']['p']:.6f}")

print("Permutation test")
result = permutation_interaction_test(df_batch, 'score', 'model_name', 'map_usage_type')
print(f"Interaction test: stat = {result['statistic']:.3f}, p = {result['p_value']:.6f}")

print("Ranking consistency test")
comparison_results, rankings = test_ranking_consistency(
    df_batch, 'score', 'model_name', 'map_usage_type'
)
print("\nRanking consistency:")
print(comparison_results)
print("\nRankings per category:")
print(rankings)

print("Simple effects analysis:")
interaction_results_c1 = []
for category in sorted(df_batch['map_usage_type'].unique()):
    subset = df_batch[df_batch['map_usage_type'] == category]
    model_groups_cat = [subset[subset['model_name'] == model]['score'].values
                        for model in sorted(subset['model_name'].unique())]
    h, p = kruskal(*model_groups_cat)
    interaction_results_c1.append({
        'category': category,
        'H': h,
        'p': p,
        'significant': p < 0.05
    })
    print(f"\n{category}: H = {h:.3f}, p = {p:.6f} {'*' if p < 0.05 else ''}")

    if p < 0.05:
        cat_means = subset.groupby('model_name')['score'].mean().sort_values(ascending=False)
        print(f"  Top 3 models: {', '.join(cat_means.head(3).index.tolist())}")

# FDR correction across categories
p_values_c1 = [r['p'] for r in interaction_results_c1]
rejected, p_corrected, _, _ = multipletests(p_values_c1, alpha=0.05, method='fdr_bh')

print("\n\nFDR-corrected results:")
for i, result in enumerate(interaction_results_c1):
    result['p_corrected'] = p_corrected[i]
    result['significant_corrected'] = rejected[i]
    print(f"{result['category']}: p_corrected = {p_corrected[i]:.6f} {'*' if rejected[i] else ''}")

# Interaction heatmap data
interaction_matrix_c1 = df_batch.pivot_table(
    values='score',
    index='model_name',
    columns='map_usage_type',
    aggfunc='mean'
)
print("\n\nMean scores (Model × Map Usage Type):")
print(interaction_matrix_c1.round(3))

# Calculate performance range per model
performance_range = interaction_matrix_c1.max(axis=1) - interaction_matrix_c1.min(axis=1)
performance_range_df = pd.DataFrame({
    'model_name': performance_range.index,
    'range': performance_range.values,
    'min_category': interaction_matrix_c1.idxmin(axis=1),
    'max_category': interaction_matrix_c1.idxmax(axis=1)
}).sort_values(by='range', ascending=False)

print("\nPerformance range per model (max mean - min mean):")
for _, row in performance_range_df.iterrows():
    print(f"{row['model_name']}: range = {row['range']:.2f} "
          f"(worst: {row['min_category']}, best: {row['max_category']})")

I1: MODEL × MAP USAGE TYPE INTERACTION
--------------------------------------------------------------------------------
Scheirer-Ray-Hare test:
Model effect: H = 90.453, p = 0.000000
Usage type effect: H = 15.191, p = 0.000503
Interaction: H = 1621.356, p = 0.000000
Permutation test
Interaction test: stat = 127.843, p = 0.000000
Ranking consistency test

Ranking consistency:
        category1       category2  kendall_tau   p_value agreement
0        analysis  interpretation     0.242424  0.310810       low
1        analysis         reading     0.484848  0.031050  moderate
2  interpretation         reading     0.090909  0.737306       low

Rankings per category:
map_usage_type        analysis  interpretation  reading
model_name                                             
Claude 3.5 Sonnet v2       2.0             7.0      3.0
Claude 3.7 Sonnet          1.0             1.0      1.0
DeepSeek-R1               11.0            11.0      8.0
GPT o3                     3.0             4.0    

In [27]:
# I2: Graphical complexity x Map Usage Type
# H0: There are no significant effects of map graphical complexity or map usage type on model performance, and no interaction between these factors.
# H1: There is a significant interaction between graphical complexity and map usage type.

print("I2: GRAPHICAL COMPLEXITY × MAP USAGE TYPE")
print("-" * 80)

# ===== 1. MAIN EFFECTS =====

df_c5 = df_batch[['score', 'graphical_complexity', 'map_usage_type']].copy()

df_c5['rank'] = df_c5['score'].rank()

model = ols('rank ~ C(graphical_complexity) + C(map_usage_type) + C(graphical_complexity):C(map_usage_type)',
            data=df_c5).fit()
anova_table = anova_lm(model, typ=2)

N = len(df_c5)
anova_table['H'] = anova_table['F'] * (anova_table['df'] + 1)
anova_table['p_srh'] = chi2.sf(anova_table['H'], anova_table['df'])

print("Scheirer-Ray-Hare Test Results:")
print(f"Complexity effect: H = {anova_table.loc['C(graphical_complexity)', 'H']:.3f}, " +
      f"p = {anova_table.loc['C(graphical_complexity)', 'p_srh']:.6f}")
print(f"Usage type effect: H = {anova_table.loc['C(map_usage_type)', 'H']:.3f}, " +
      f"p = {anova_table.loc['C(map_usage_type)', 'p_srh']:.6f}")
print(f"Interaction: H = {anova_table.loc['C(graphical_complexity):C(map_usage_type)', 'H']:.3f}, " +
      f"p = {anova_table.loc['C(graphical_complexity):C(map_usage_type)', 'p_srh']:.6f}")

interaction_p = anova_table.loc['C(graphical_complexity):C(map_usage_type)', 'p_srh']
print(f"\nInteraction significant: {'YES' if interaction_p < 0.05 else 'NO'}")

# ===== 2. SIMPLE EFFECTS =====
if interaction_p < 0.05:
    print("\n" + "="*80)
    print("Simple effects analysis (post-hoc):")
    interaction_results_c5 = []

    for category in sorted(df_batch['map_usage_type'].unique()):
        subset = df_batch[df_batch['map_usage_type'] == category]
        comp_groups = [subset[subset['graphical_complexity'] == comp]['score'].values
                       for comp in sorted(subset['graphical_complexity'].unique())]
        u, p = mannwhitneyu(comp_groups[0], comp_groups[1], alternative='two-sided')

        # Effect size
        n1, n2 = len(comp_groups[0]), len(comp_groups[1])
        r = 1 - (2*u) / (n1 * n2)  # rank-biserial

        interaction_results_c5.append({
            'category': category,
            'U': u,
            'p': p,
            'r': r,
            'significant': p < 0.05
        })
        print(f"\n{category}: U = {u:.3f}, p = {p:.6f}, r = {r:.3f} {'*' if p < 0.05 else ''}")

    # FDR correction
    p_values_c5 = [r['p'] for r in interaction_results_c5]
    rejected_c5, p_corrected_c5, _, _ = multipletests(p_values_c5, alpha=0.05, method='fdr_bh')

    print("\n\nFDR-corrected results:")
    for i, result in enumerate(interaction_results_c5):
        result['p_corrected'] = p_corrected_c5[i]
        result['significant_corrected'] = rejected_c5[i]
        print(f"{result['category']}: p_corrected = {p_corrected_c5[i]:.6f} {'*' if rejected_c5[i] else ''}")
else:
    print("\nInteraction not significant - simple effects analysis not warranted")
    print("Main effect of complexity applies uniformly across usage types")

# ===== 3. DESCRIPTIVE STATISTICS =====
print("\n\nMean scores (Complexity × Category):")
interaction_matrix_c5 = df_batch.pivot_table(
    values='score',
    index='graphical_complexity',
    columns='map_usage_type',
    aggfunc='mean'
)
print(interaction_matrix_c5.round(3))

print("\n\nEffect of low complexity (difference from high):")
for col in interaction_matrix_c5.columns:
    diff = interaction_matrix_c5.loc['low', col] - interaction_matrix_c5.loc['high', col]
    print(f"{col}: +{diff:.3f}")

I2: GRAPHICAL COMPLEXITY × MAP USAGE TYPE
--------------------------------------------------------------------------------
Scheirer-Ray-Hare Test Results:
Complexity effect: H = 44.492, p = 0.000000
Usage type effect: H = 23.313, p = 0.000009
Interaction: H = 10.667, p = 0.004827

Interaction significant: YES

Simple effects analysis (post-hoc):

analysis: U = 34892.500, p = 0.000812, r = 0.159 *

interpretation: U = 39849.500, p = 0.415771, r = 0.039 

reading: U = 34900.500, p = 0.000514, r = 0.158 *


FDR-corrected results:
analysis: p_corrected = 0.001218 *
interpretation: p_corrected = 0.415771 
reading: p_corrected = 0.001218 *


Mean scores (Complexity × Category):
map_usage_type        analysis  interpretation  reading
graphical_complexity                                   
high                     2.678           3.673    2.441
low                      3.388           3.713    3.219


Effect of low complexity (difference from high):
analysis: +0.710
interpretation: +0.040
read

In [28]:
# I3: Model x Graphical complexity
# H0: There is no significant effect of the model, no effect of graphical complexity, and no interaction between model and graphical complexity.
# H1: There is a significant interaction between model and graphical complexity.

print("I3: MODEL × GRAPHICAL COMPLEXITY INTERACTION")
print("-" * 80)

results = scheirer_ray_hare_test(df_batch, 'score', 'model_name', 'graphical_complexity')
print(f"Model effect: H = {results['model_effect']['H']:.3f}, p = {results['model_effect']['p']:.6f}")
print(f"Graphical complexity effect: H = {results['graphical_complexity_effect']['H']:.3f}, p = {results['graphical_complexity_effect']['p']:.6f}")
print(f"Interaction: H = {results['interaction']['H']:.3f}, p = {results['interaction']['p']:.6f}")

print("Simple effects analysis:")
interaction_results_c2 = []
for complexity in sorted(df_batch['graphical_complexity'].unique()):
    subset = df_batch[df_batch['graphical_complexity'] == complexity]
    model_groups_comp = [subset[subset['model_name'] == model]['score'].values
                         for model in sorted(subset['model_name'].unique())]
    h, p = kruskal(*model_groups_comp)
    interaction_results_c2.append({
        'graphical_complexity': complexity,
        'H': h,
        'p': p,
        'significant': p < 0.05
    })
    print(f"\n{complexity}: H = {h:.3f}, p = {p:.6f} {'*' if p < 0.05 else ''}")

    if p < 0.05:
        comp_means = subset.groupby('model_name')['score'].mean().sort_values(ascending=False)
        print(f"  Top 3 models: {', '.join(comp_means.head(3).index.tolist())}")

p_values_c2 = [r['p'] for r in interaction_results_c2]
rejected_c2, p_corrected_c2, _, _ = multipletests(p_values_c2, alpha=0.05, method='fdr_bh')

print("\n\nFDR-corrected results:")
for i, result in enumerate(interaction_results_c2):
    result['p_corrected'] = p_corrected_c2[i]
    result['significant_corrected'] = rejected_c2[i]
    print(f"{result['graphical_complexity']}: p_corrected = {p_corrected_c2[i]:.6f} {'*' if rejected_c2[i] else ''}")

interaction_matrix_c2 = df_batch.pivot_table(
    values='score',
    index='model_name',
    columns='graphical_complexity',
    aggfunc='mean'
)
interaction_matrix_c2['low_minus_high'] = (
    interaction_matrix_c2['low'] - interaction_matrix_c2['high']
)

print("\n\nMean scores (Model × Graphical Complexity):")
print(interaction_matrix_c2.round(3))

I3: MODEL × GRAPHICAL COMPLEXITY INTERACTION
--------------------------------------------------------------------------------
Model effect: H = 90.453, p = 0.000000
Graphical complexity effect: H = 21.744, p = 0.000003
Interaction: H = 1614.803, p = 0.000000
Simple effects analysis:

high: H = 34.727, p = 0.000275 *
  Top 3 models: Claude 3.7 Sonnet, Claude 3.5 Sonnet v2, GPT o3

low: H = 72.624, p = 0.000000 *
  Top 3 models: Claude 3.7 Sonnet, Mistral Large, GPT o3


FDR-corrected results:
high: p_corrected = 0.000275 *
low: p_corrected = 0.000000 *


Mean scores (Model × Graphical Complexity):
graphical_complexity   high    low  low_minus_high
model_name                                        
Claude 3.5 Sonnet v2  3.425  3.729           0.304
Claude 3.7 Sonnet     3.580  4.291           0.711
DeepSeek-R1           2.419  3.184           0.765
GPT o3                3.241  3.785           0.544
GPT-4o                2.815  3.142           0.327
Gemini 1.5 Pro        3.134  3.442     

In [29]:
# I4: Model x Spatial aggregation level
# H0: There is no significant effect of the model, no effect of NUTS level, and no interaction between model and NUTS level.
# H1: There is a significant interaction between model and NUTS level.

print("I4: MODEL × SPATIAL AGGREGATION LEVEL INTERACTION")
print("-" * 80)

results = scheirer_ray_hare_test(df_batch, 'score', 'model_name', 'nuts_level')
print(f"Model effect: H = {results['model_effect']['H']:.3f}, p = {results['model_effect']['p']:.6f}")
print(f"NUTS level effect: H = {results['nuts_level_effect']['H']:.3f}, p = {results['nuts_level_effect']['p']:.6f}")
print(f"Interaction: H = {results['interaction']['H']:.3f}, p = {results['interaction']['p']:.6f}")

print("Simple effects analysis:")
interaction_results_c3 = []
for nuts in sorted(df_batch['nuts_level'].unique()):
    subset = df_batch[df_batch['nuts_level'] == nuts]
    model_groups_nuts = [subset[subset['model_name'] == model]['score'].values
                         for model in sorted(subset['model_name'].unique())]
    h, p = kruskal(*model_groups_nuts)
    interaction_results_c3.append({
        'nuts_level': nuts,
        'H': h,
        'p': p,
        'significant': p < 0.05
    })
    print(f"\n{nuts}: H = {h:.3f}, p = {p:.6f} {'*' if p < 0.05 else ''}")

    if p < 0.05:
        nuts_means = subset.groupby('model_name')['score'].mean().sort_values(ascending=False)
        print(f"  Top 3 models: {', '.join(nuts_means.head(3).index.tolist())}")

# FDR correction across NUTS levels
p_values_c3 = [r['p'] for r in interaction_results_c3]
rejected_c3, p_corrected_c3, _, _ = multipletests(p_values_c3, alpha=0.05, method='fdr_bh')

print("\n\nFDR-corrected results:")
for i, result in enumerate(interaction_results_c3):
    result['p_corrected'] = p_corrected_c3[i]
    result['significant_corrected'] = rejected_c3[i]
    print(f"{result['nuts_level']}: p_corrected = {p_corrected_c3[i]:.6f} {'*' if rejected_c3[i] else ''}")

interaction_matrix_c3 = df_batch.pivot_table(
    values='score',
    index='model_name',
    columns='nuts_level',
    aggfunc='mean'
)
interaction_matrix_c3['country_minus_region'] = (
    interaction_matrix_c3['country'] - interaction_matrix_c3['region']
)
print("\n\nMean scores (Model × NUTS):")
print(interaction_matrix_c3.round(3))

I4: MODEL × SPATIAL AGGREGATION LEVEL INTERACTION
--------------------------------------------------------------------------------
Model effect: H = 90.453, p = 0.000000
NUTS level effect: H = 3.653, p = 0.055958
Interaction: H = 1632.894, p = 0.000000
Simple effects analysis:

country: H = 53.150, p = 0.000000 *
  Top 3 models: Claude 3.7 Sonnet, Claude 3.5 Sonnet v2, GPT o3

region: H = 42.932, p = 0.000011 *
  Top 3 models: Claude 3.7 Sonnet, GPT o3, Claude 3.5 Sonnet v2


FDR-corrected results:
country: p_corrected = 0.000000 *
region: p_corrected = 0.000011 *


Mean scores (Model × NUTS):
nuts_level            country  region  country_minus_region
model_name                                                 
Claude 3.5 Sonnet v2    3.690   3.464                 0.226
Claude 3.7 Sonnet       4.097   3.774                 0.323
DeepSeek-R1             2.838   2.765                 0.073
GPT o3                  3.535   3.491                 0.044
GPT-4o                  3.057   2.899  

## 5. Clusters

In [30]:
print("P1: CLUSTERS")
print("-" * 80)

df_d3 = df_batch.copy()

df_d3['difficulty_combo'] = (df_d3['graphical_complexity'] + '_' +
                             df_d3['nuts_level'] + '_' +
                             df_d3['map_source'] + '_' +
                             df_d3['map_usage_type'])

combo_counts = df_d3['difficulty_combo'].value_counts()

valid_combos = combo_counts[combo_counts >= 10].index
df_d3 = df_d3[df_d3['difficulty_combo'].isin(valid_combos)]

profile_analysis = df_d3.groupby(['model_name', 'difficulty_combo'])['score'].agg([
    'mean', 'std', 'count'
]).unstack(fill_value=np.nan)

# Cluster analysis of models based on difficulty profiles
try:
    # Prepare data for clustering
    model_profiles = profile_analysis['mean'].fillna(profile_analysis['mean'].mean(axis=0))

    # Standardize
    scaler = StandardScaler()
    profiles_scaled = scaler.fit_transform(model_profiles)

    # Determine optimal number of clusters (2-5)
    silhouette_scores = {}
    for n_clusters in range(2, min(6, len(model_profiles))):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        cluster_labels = kmeans.fit_predict(profiles_scaled)
        silhouette_scores[n_clusters] = silhouette_score(profiles_scaled, cluster_labels)

    optimal_k = max(silhouette_scores, key=silhouette_scores.get)
    print(f"\n\nCluster Analysis:")
    print(f"Silhouette scores for different k: {silhouette_scores}")
    print(f"Optimal number of clusters: {optimal_k}")

    # K-means clustering with optimal k
    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(profiles_scaled)

    # Assign clusters to models
    model_clusters = dict(zip(model_profiles.index, clusters))

    print(f"\nModel cluster assignments (k={optimal_k}):")
    for cluster_id in range(optimal_k):
        cluster_models = [m for m, c in model_clusters.items() if c == cluster_id]
        print(f"  Cluster {cluster_id}: {', '.join(cluster_models)}")

    # Characterize clusters by their performance patterns
    cluster_characteristics = {}
    for cluster_id in range(optimal_k):
        cluster_models = [m for m, c in model_clusters.items() if c == cluster_id]
        cluster_data = df_d3[df_d3['model_name'].isin(cluster_models)]

        cluster_characteristics[cluster_id] = {
            'mean_score': cluster_data['score'].mean(),
            'std_score': cluster_data['score'].std(),
            'best_on_low': cluster_data[cluster_data['graphical_complexity'] == 'low']['score'].mean(),
            'best_on_high': cluster_data[cluster_data['graphical_complexity'] == 'high']['score'].mean(),
            'models': cluster_models
        }

    print("\nCluster characteristics:")
    for cluster_id, chars in cluster_characteristics.items():
        print(f"\n  Cluster {cluster_id}:")
        print(f"    Overall: M={chars['mean_score']:.3f}, SD={chars['std_score']:.3f}")
        print(f"    Low graphically complex maps: M={chars['best_on_low']:.3f}")
        print(f"    High graphically complex maps: M={chars['best_on_high']:.3f}")

    cluster_analysis = {
        'model_clusters': model_clusters,
        'optimal_k': optimal_k,
        'silhouette_scores': silhouette_scores,
        'cluster_characteristics': cluster_characteristics
    }

except ImportError:
    print("\n\nSklearn not available - skipping cluster analysis")
    cluster_analysis = None

# Statistical tests comparing clusters (not individual models)
if cluster_analysis is not None:
    print("\n\nStatistical tests comparing clusters per difficulty combination:")
    
    # Add cluster assignment to dataframe
    df_d3['cluster'] = df_d3['model_name'].map(cluster_analysis['model_clusters'])
    
    cluster_anova_results = {}
    cluster_p_values = []
    cluster_combo_names = []
    
    for combo in valid_combos:
        combo_data = df_d3[df_d3['difficulty_combo'] == combo]
        unique_clusters = combo_data['cluster'].dropna().unique()
        
        if len(unique_clusters) > 1:
            # Kruskal-Wallis for comparing clusters
            cluster_groups = [combo_data[combo_data['cluster'] == cluster_id]['score'].values
                             for cluster_id in unique_clusters]
            h_stat, p_val = kruskal(*cluster_groups)
            
            n = len(combo_data)
            k = len(unique_clusters)
            epsilon_sq = (h_stat - k + 1) / (n - k)
            
            cluster_anova_results[combo] = {
                'test': 'Kruskal-Wallis',
                'statistic': h_stat,
                'p_value': p_val,
                'effect_size': epsilon_sq,
                'significant': p_val < 0.05,
                'n_clusters': len(unique_clusters),
                'n_obs': n
            }
            cluster_p_values.append(p_val)
            cluster_combo_names.append(combo)
    
    print("\nCombinations significant before FDR (trends):")
    for combo, result in cluster_anova_results.items():
        if result['significant'] and not result.get('significant_corrected', False):
            print(f"  {combo}: H={result['statistic']:.2f}, p={result['p_value']:.4f}, ε²={result['effect_size']:.3f}")
    
    # FDR correction across all combinations
    if len(cluster_p_values) > 0:
        rejected, p_corrected, _, _ = multipletests(cluster_p_values, alpha=0.05, method='fdr_bh')
        
        print(f"\nTotal combinations tested: {len(cluster_p_values)}")
        print(f"Significant before correction: {sum([r['significant'] for r in cluster_anova_results.values()])}")
        print(f"Significant after FDR correction: {sum(rejected)}")
        
        # Update results with corrected p-values
        for i, combo in enumerate(cluster_combo_names):
            cluster_anova_results[combo]['p_corrected'] = p_corrected[i]
            cluster_anova_results[combo]['significant_corrected'] = rejected[i]
        
        # Show significant results after correction
        print("\nSignificant differences between clusters after FDR correction:")
        sig_count = 0
        for combo, result in cluster_anova_results.items():
            if result.get('significant_corrected', False):
                sig_count += 1
                print(f"\n  {combo}:")
                print(f"    H={result['statistic']:.3f}, p_corr={result['p_corrected']:.6f}, ε²={result['effect_size']:.3f}")
                # Show cluster means for this combination
                combo_cluster_means = df_d3[df_d3['difficulty_combo'] == combo].groupby('cluster')['score'].mean().sort_values(ascending=False)
                print(f"    Cluster means:")
                for cluster_id, mean_score in combo_cluster_means.items():
                    cluster_models = [m for m, c in cluster_analysis['model_clusters'].items() if c == cluster_id]
                    print(f"      Cluster {int(cluster_id)} ({', '.join(cluster_models)}): M={mean_score:.3f}")
        
        if sig_count == 0:
            print("  None - no significant differences between clusters after correction")
else:
    print("\n\nCluster analysis not available - skipping statistical tests comparing clusters")

P1: CLUSTERS
--------------------------------------------------------------------------------


Cluster Analysis:
Silhouette scores for different k: {2: 0.1617581229454909, 3: 0.1462751206484514, 4: 0.14309861286186268, 5: 0.1609622264512788}
Optimal number of clusters: 2

Model cluster assignments (k=2):
  Cluster 0: DeepSeek-R1, GPT-4o, Gemma 3, MiniMax-01, Qwen2.5-Max, Sonar
  Cluster 1: Claude 3.5 Sonnet v2, Claude 3.7 Sonnet, GPT o3, Gemini 1.5 Pro, Grok-3, Mistral Large

Cluster characteristics:

  Cluster 0:
    Overall: M=2.872, SD=2.040
    Low graphically complex maps: M=3.084
    High graphically complex maps: M=2.660

  Cluster 1:
    Overall: M=3.499, SD=1.895
    Low graphically complex maps: M=3.796
    High graphically complex maps: M=3.201


Statistical tests comparing clusters per difficulty combination:

Combinations significant before FDR (trends):
  low_region_statistical_office_analysis: H=14.10, p=0.0002, ε²=0.092
  low_region_statistical_office_reading: H=10.38,

In [31]:
# Check missing combinations
df_batch[
    (df_batch['graphical_complexity'] == 'high') &
    (df_batch['nuts_level'] == 'region') &
    (df_batch['map_source'] == 'statistical_office')
].shape[0]

0

In [33]:
if cluster_analysis is not None and 'model_clusters' in cluster_analysis:
    cluster_assignments = cluster_analysis['model_clusters']
    print("Using cluster assignments from P1 analysis")
    print(f"Clusters: {cluster_analysis['optimal_k']}")
    for cluster_id in range(cluster_analysis['optimal_k']):
        cluster_models = [m for m, c in cluster_assignments.items() if c == cluster_id]
        print(f"  Cluster {cluster_id}: {', '.join(cluster_models)}")
else:
    cluster_assignments = {
        'DeepSeek-R1': 0, 'GPT-4o': 0, 'Gemma 3': 0,
        'MiniMax-01': 0, 'Qwen2.5-Max': 0, 'Sonar': 0,
        'Claude 3.5 Sonnet v2': 1, 'Claude 3.7 Sonnet': 1,
        'GPT o3': 1, 'Gemini 1.5 Pro': 1, 'Grok-3': 1, 'Mistral Large': 1
    }
    print("Using hardcoded cluster assignments (cluster_analysis not available)")

df_batch['cluster'] = df_batch['model_name'].map(cluster_assignments)

categories = ['reading', 'analysis', 'interpretation']

unique_clusters = sorted(df_batch['cluster'].dropna().unique())
n_clusters = len(unique_clusters)

print(f"\nAnalyzing {n_clusters} clusters")

cluster_data = {cat: {} for cat in categories}

for cat in categories:
    for cluster_id in unique_clusters:
        cluster_data[cat][f'cluster_{int(cluster_id)}_mean'] = []
        cluster_data[cat][f'cluster_{int(cluster_id)}_sem'] = []

for cat in categories:
    for cluster_id in unique_clusters:
        c_data = df_batch[(df_batch['cluster'] == cluster_id) & (df_batch['map_usage_type'] == cat)]['score']
        cluster_data[cat][f'cluster_{int(cluster_id)}_mean'].append(c_data.mean())
        cluster_data[cat][f'cluster_{int(cluster_id)}_sem'].append(c_data.sem())

df_dict = {'category': categories}
for cluster_id in unique_clusters:
    df_dict[f'cluster_{int(cluster_id)}_mean'] = [cluster_data[cat][f'cluster_{int(cluster_id)}_mean'][0] for cat in categories]
    df_dict[f'cluster_{int(cluster_id)}_sem'] = [cluster_data[cat][f'cluster_{int(cluster_id)}_sem'][0] for cat in categories]

cluster_performance_df = pd.DataFrame(df_dict)

output_file = "../../results/cluster_performance_summary.csv"
cluster_performance_df.to_csv(output_file, index=False)

Using cluster assignments from P1 analysis
Clusters: 2
  Cluster 0: DeepSeek-R1, GPT-4o, Gemma 3, MiniMax-01, Qwen2.5-Max, Sonar
  Cluster 1: Claude 3.5 Sonnet v2, Claude 3.7 Sonnet, GPT o3, Gemini 1.5 Pro, Grok-3, Mistral Large

Analyzing 2 clusters
